In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sklearn
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV

In [3]:
# Data Paths (NE RIEN CHANGER)
input_dir = '/kaggle/input/le-titanic-spatial'
output_dir = '/kaggle/working'

## Définition de fonctions

On définit, ici, pleins de fonctions qui pourraient nous être utile pour l'entraînement des modèles. 

Cela permet de simplifier le processur d'entraînement. Normalement, on placerait plutôt ces fonctions dans un fichier python. Mais pour simplifier, elles sont toutes disponibles dans ce notebook.

In [4]:
def load_data(input_dir: str):
    """
    Charge les données de fichiers CSV de formation et de test.

    Paramètres
    ----------
    input_dir : str
        Le répertoire contenant les fichiers 'train.csv' et 'test.csv'.

    Retourne
    -------
    tuple
        Un tuple contenant deux DataFrames pandas : (train, test).
    """

    train = pd.read_csv(os.path.join(input_dir, 'train.csv'))
    test = pd.read_csv(os.path.join(input_dir, 'test.csv'))
    return train, test

def prepare_data(train: pd.DataFrame, test:pd.DataFrame):
    """
    Prépare les données d'entraînement et de test en ajoutant des transformations et des informations supplémentaires.

    Paramètres
    ----------
    train : pd.DataFrame
        Le DataFrame contenant les données d'entraînement.
    test : pd.DataFrame
        Le DataFrame contenant les données de test.

    Retours
    -------
    tuple
        Un tuple contenant les DataFrames d'entraînement et de test modifiés.

    Notes
    -----
    - Ajoute les colonnes log-transformées pour 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', et 'VRDeck'.
    - Ajoute des informations sur les cabines en séparant la colonne 'Cabin' en 'Deck', 'Num', et 'Side'.
    """


    # Add log to some columns
    num_cols_log = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

    # Si on rajoute de nouvelles features dans le train set, on les rajoute directement dans le test set
    for col in num_cols_log:
        train[col + '_log'] = np.log(train[col] + 1)
        test[col + '_log'] = np.log(test[col] + 1)

    # Add Cabin information
    splitted_values_train = train['Cabin'].apply(split_cabin)
    splitted_values_test = test['Cabin'].apply(split_cabin)

    # Deck
    train['Deck'] = splitted_values_train.apply(lambda x: x[0])
    test['Deck'] = splitted_values_test.apply(lambda x: x[0])

    # Num
    train['Num'] = splitted_values_train.apply(lambda x: x[1])
    test['Num'] = splitted_values_test.apply(lambda x: x[1])

    # Side
    train['Side'] = splitted_values_train.apply(lambda x: x[2])
    test['Side'] = splitted_values_test.apply(lambda x: x[2])

    return train, test

def split_cabin(cabin_encoded: str):
    """
    Divise une chaîne de caractères représentant une cabine en trois parties distinctes.
    Parameters
    ----------
    cabin_encoded : str
        La chaîne de caractères encodée représentant la cabine.
    Returns
    -------
    list
        Une liste contenant trois éléments : 
        - Le premier élément est une chaîne de caractères représentant la première partie de la cabine.
        - Le deuxième élément est un entier représentant la deuxième partie de la cabine.
        - Le troisième élément est une chaîne de caractères représentant la troisième partie de la cabine.
        Si `cabin_encoded` est NaN, retourne [np.nan, np.nan, np.nan].
    """

    if pd.isna(cabin_encoded):
        return [np.nan, np.nan, np.nan]
    
    else:

        splitted_values = cabin_encoded.split('/')

        # La valeur du milieu est un chiffre, on va donc le "caster" directement

        return splitted_values[0], int(splitted_values[1]), splitted_values[2]
    
def apply_preprocessing(train: pd.DataFrame, test: pd.DataFrame):
    """
    Applique le prétraitement aux ensembles de données d'entraînement et de test.
    Cette fonction prend en entrée deux DataFrames, `train` et `test`, et applique des transformations
    numériques et catégorielles aux caractéristiques spécifiées. Elle renvoie les ensembles de données
    prétraités ainsi que la cible d'entraînement.
    Parameters
    ----------
    train : pd.DataFrame
        Le DataFrame contenant les données d'entraînement.
    test : pd.DataFrame
        Le DataFrame contenant les données de test.
    Returns
    -------
    features_train_preprocessed : np.ndarray
        Les caractéristiques d'entraînement après prétraitement.
    features_test_preprocessed : np.ndarray
        Les caractéristiques de test après prétraitement.
    target_train : pd.Series
        La cible d'entraînement.
    """
    
    num_features = ['Age', 'RoomService_log', 'FoodCourt_log', 'ShoppingMall_log', 'Spa_log', 'VRDeck_log', 'Num']
    cat_features = ['HomePlanet', 'Deck', 'Side', 'Destination']
    # Les valeurs booléennes sont que des 0 et des 1 => pas besoin de faire de transformation
    bool_features = ['VIP', 'CryoSleep']
    target = 'Transported'

    features_train = train[num_features + cat_features + bool_features]
    features_test = test[num_features + cat_features + bool_features]
    target_train = train[target]

    # Numerical transformer
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])

    # Categorical transformer
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Le ColumnTransformer est une pipeline qui me permet d'appliquer les transformations sur tous les datasets que je peux avoir à utiliser
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, num_features),
            ('cat', categorical_transformer, cat_features)
        ])
    
    features_train_preprocessed = preprocessor.fit_transform(features_train)
    features_test_preprocessed = preprocessor.transform(features_test)

    return features_train_preprocessed, features_test_preprocessed, target_train

def prepare_kaggle_submission(file_name, test, predictions):
    """
    Prépare un fichier de soumission pour Kaggle.

    Parameters
    ----------
    file_name : str
        Le nom du fichier de soumission à créer.
    test : pandas.DataFrame
        Le DataFrame contenant les données de test, incluant la colonne 'PassengerId'.
    predictions : array-like
        Les prédictions du modèle à soumettre, correspondant aux données de test.

    Returns
    -------
    None
        Cette fonction ne retourne rien. Elle crée un fichier CSV de soumission.
    """


    print(f"Création du fichier de soumission {file_name}")
    submission = pd.DataFrame({'PassengerId': test.PassengerId, 'Transported': predictions})

    submission['Transported'] = submission['Transported'].astype(bool)

    submission.to_csv(os.path.join(output_dir, file_name), index=False)

def train_model_all_data_and_predict(model, features_train, target_train, features_test):
    """
    Entraîne un modèle sur toutes les données d'entraînement et effectue des prédictions sur les données de test.
    Parameters
    ----------
    model : object
        Le modèle à entraîner. Doit implémenter les méthodes `fit` et `predict`.
    features_train : array-like
        Les caractéristiques des données d'entraînement.
    target_train : array-like
        Les cibles des données d'entraînement.
    features_test : array-like
        Les caractéristiques des données de test.
    Returns
    -------
    predictions : array-like
        Les prédictions du modèle sur les données de test.
    """

    model.fit(features_train, target_train)
    predictions = model.predict(features_test)

    return predictions

def do_cross_validation(model, features_train, target_train, cv=5):
    """
    Effectue une validation croisée sur le modèle donné.
    Paramètres
    ----------
    model : estimator
        Le modèle à valider.
    features_train : array-like
        Les caractéristiques d'entraînement.
    target_train : array-like
        Les cibles d'entraînement.
    cv : int, optionnel
        Le nombre de plis pour la validation croisée (par défaut 5).
    Affiche
    -------
    Cross-validation scores : array
        Les scores de validation croisée pour chaque pli.
    Mean CV accuracy : float
        La précision moyenne de la validation croisée.
    """

    scores = cross_val_score(model, features_train, target_train, cv=cv, scoring='accuracy')
    print(f"Cross-validation scores: {scores}")
    print(f"Mean CV accuracy: {scores.mean():.2%}")

def test_for_overfitting(model, features_train, target_train, test_size=0.2):
    """
    Teste le surapprentissage d'un modèle en comparant les performances sur les ensembles
    d'entraînement et de test.
    Parameters
    ----------
    model : object
        Le modèle à entraîner et à tester. Doit implémenter les méthodes `fit` et `predict`.
    features_train : array-like
        Les caractéristiques d'entraînement.
    target_train : array-like
        Les cibles d'entraînement.
    test_size : float, optional
        La proportion de l'ensemble de données à inclure dans l'ensemble de test (default is 0.2).
    Returns
    -------
    None
        Cette fonction ne retourne rien. Elle affiche les rapports de classification pour les ensembles
        d'entraînement et de test.
    """


    X_train, X_test, y_train, y_test = train_test_split(features_train, target_train, test_size=test_size)

    model.fit(X_train, y_train)

    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    print("Résultats sur le train set")
    print(classification_report(y_train, y_pred_train))
    print()
    print("Résultats sur le test set")
    print(classification_report(y_test, y_pred_test))

def tune_hyperparameters(model, features_train, target_train, param_distributions, cv=5, n_iter=20):
    """
    Tune les hyperparamètres d'un modèle en utilisant une recherche aléatoire.
    Cette fonction utilise RandomizedSearchCV pour effectuer une recherche aléatoire
    sur les hyperparamètres spécifiés et trouver la meilleure combinaison pour le modèle donné.
    Parameters
    ----------
    model : estimator object
        Le modèle pour lequel les hyperparamètres doivent être optimisés.
    features_train : array-like or DataFrame
        Les caractéristiques d'entraînement.
    target_train : array-like or Series
        La cible d'entraînement.
    param_distributions : dict
        Le dictionnaire contenant les distributions des hyperparamètres à tester.
    cv : int, default=5
        Le nombre de folds pour la validation croisée.
    n_iter : int, default=100
        Le nombre d'itérations pour la recherche aléatoire.
    Returns
    -------
    best_estimator_ : estimator object
        Le modèle avec les meilleurs hyperparamètres trouvés.
    Examples
    --------
    >>> from sklearn.ensemble import RandomForestClassifier
    >>> param_distributions = {
    ...     'n_estimators': [100, 200, 300],
    ...     'max_depth': [None, 10, 20, 30]
    ... }
    >>> model = RandomForestClassifier()
    >>> best_model = tune_hyperparameters(model, X_train, y_train, param_distributions)
    Best params: {'max_depth': 20, 'n_estimators': 200}
    Best score: 0.85
    """

    random_search = RandomizedSearchCV(model, param_distributions=param_distributions, n_iter=n_iter, cv=cv, scoring='accuracy', n_jobs=-1, verbose=2)
    random_search.fit(features_train, target_train)

    print(f"Best params: {random_search.best_params_}")
    print(f"Best score: {random_search.best_score_}")

    return random_search.best_estimator_

## Préparation des données

On utilise les fonctions disponibles pour préparer les données afin de pouvoir entraîner les modèles. 

In [5]:
train, test = load_data(input_dir)

train, test = prepare_data(train, test)

features_train_preprocessed, features_test_preprocessed, target_train = apply_preprocessing(train, test)

## Entraînement d'un RandomForest

In [6]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42)

# On fait une CV pour savoir si notre modèle prédit correctement
do_cross_validation(model, features_train_preprocessed, target_train)

Cross-validation scores: [0.7533065  0.7573318  0.79413456 0.81875719 0.78711162]
Mean CV accuracy: 78.21%


In [7]:
# On peut tester si notre modèle overfit
test_for_overfitting(model, features_train_preprocessed, target_train)

Résultats sur le train set
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      3443
        True       1.00      1.00      1.00      3511

    accuracy                           1.00      6954
   macro avg       1.00      1.00      1.00      6954
weighted avg       1.00      1.00      1.00      6954


Résultats sur le test set
              precision    recall  f1-score   support

       False       0.79      0.83      0.81       872
        True       0.82      0.77      0.80       867

    accuracy                           0.80      1739
   macro avg       0.80      0.80      0.80      1739
weighted avg       0.80      0.80      0.80      1739



In [8]:
parameters = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Avec un RandomSearch (pour éviter de chercher tous les paramètres), on peut trouver les meilleurs paramètres
best_model = tune_hyperparameters(model, features_train_preprocessed, target_train, parameters)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[CV] END max_depth=30, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   3.1s
[CV] END max_depth=30, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   3.1s
[CV] END max_depth=30, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   3.3s
[CV] END max_depth=30, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   3.2s
[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   2.6s
[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   2.6s
[CV] END max_depth=30, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   3.7s[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   2.8s

[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   3.0s
[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=10, n_estimators=100; tota

In [9]:
# On test le meilleur modèle pour voir s'il overfit
test_for_overfitting(best_model, features_train_preprocessed, target_train)

Résultats sur le train set
              precision    recall  f1-score   support

       False       0.86      0.84      0.85      3454
        True       0.85      0.87      0.86      3500

    accuracy                           0.86      6954
   macro avg       0.86      0.86      0.86      6954
weighted avg       0.86      0.86      0.86      6954


Résultats sur le test set
              precision    recall  f1-score   support

       False       0.80      0.81      0.80       861
        True       0.81      0.80      0.80       878

    accuracy                           0.80      1739
   macro avg       0.80      0.80      0.80      1739
weighted avg       0.80      0.80      0.80      1739



In [10]:
# Ré-entraînement du modèle et prédiction sur le test set
predictions = train_model_all_data_and_predict(best_model, features_train_preprocessed, target_train, features_test_preprocessed)

# Création du fichier de soumission
prepare_kaggle_submission('best_RF.csv', test, predictions)

Création du fichier de soumission best_RF.csv
